<a href="https://colab.research.google.com/github/vindruid/gcolab-computer-vision/blob/master/First_Order_Model_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Demo for paper "First Order Motion Model for Image Animation"

Sources :

* https://github.com/AliaksandrSiarohin/first-order-model

* https://github.com/AliaksandrSiarohin/first-order-model/blob/master/demo.ipynb

* https://papers.nips.cc/paper/8935-first-order-motion-model-for-image-animation.pdf

* https://www.youtube.com/watch?v=mUfJOQKdtAk&t=318s

---


# Download and prepare all we need 

In [0]:
# download model, video and images from google drive
import requests

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

## gdrive url -> https://drive.google.com/file/d/1-7QuciA7ZCfuNkLNMFUccot2CkdJGqqy/view?usp=sharing
file_id = '1-7QuciA7ZCfuNkLNMFUccot2CkdJGqqy'
destination = 'first-order-motion-model.zip'
download_file_from_google_drive(file_id, destination)

!unzip -qq first-order-motion-model.zip

# Clone source repo to get codes
!git clone https://github.com/AliaksandrSiarohin/first-order-model
import sys
sys.path.append('first-order-model')

# Import and define function
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML

from demo import load_checkpoints
from demo import make_animation
from skimage import img_as_ubyte

import warnings
warnings.filterwarnings("ignore")

# load model
generator, kp_detector = load_checkpoints(config_path='./first-order-model/config/vox-256.yaml', 
                            checkpoint_path='./first-order-motion-model/vox-cpk.pth.tar')

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)   
    # Animation using a fixed set of Artist objects. Before creating an instance, 
    # all plotting should have taken place and the relevant artists saved.
    plt.close()
    return ani



# Result from available image and video


Go to this [gdrive](https://drive.google.com/drive/folders/1kZ1gCnpfU0BnpdU47pLM_TQ6RypDDqgw) 
To see what images and videos available from the Author

In [0]:
source_image = imageio.imread('/content/first-order-motion-model/02.png')
driving_video = imageio.mimread('/content/first-order-motion-model/10.mp4')

#Resize image and video to 256x256
source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]


**Type 1 result**

In [0]:
predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)
HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 444/444 [00:20<00:00, 21.98it/s]


**Type 2 Result**

In [10]:
predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=False, adapt_movement_scale=True)
HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 444/444 [00:20<00:00, 22.02it/s]


# Try your own image

Upload your image into google colab

Make sure the image already croped, similar to above example

In [12]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving alberto.png to alberto.png
User uploaded file "alberto.png" with length 418105 bytes


change `source_image` with your uploaded file

In [0]:
source_image = imageio.imread('alberto.png')
driving_video = imageio.mimread('/content/first-order-motion-model/10.mp4')

#Resize image and video to 256x256
source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]


In [14]:
# using type 2 
predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=False, adapt_movement_scale=True)
HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 444/444 [00:20<00:00, 21.88it/s]


Download the video simply right click and save video

# Cheers !!! Thank You